In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import pandas as pd
import os
import numpy as np
import gensim
from gensim.models import Word2Vec, FastText
!pip install glove-python-binary
import glove
from glove import Corpus

import collections
import gc 

import warnings
warnings.filterwarnings('ignore')

In [25]:
new_notes = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/ner_df.p") # med7
w2vec = Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/embeddings/word2vec.model")
fasttext = FastText.load("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/embeddings/fasttext.model")
#word2vec = gensim.models.KeyedVectors.load('word2vec.model')

In [10]:
null_index_list = []
for i in new_notes.itertuples():
    
    if len(i.ner) == 0:
    #if not i.ner:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)
print(len(new_notes))

119519


In [11]:
med7_ner_data = {}

for ii in new_notes.itertuples():
    
    p_id = ii.SUBJECT_ID
    ind = ii.Index
    
    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []
            
    unique = set()
    new_temp = []
    
    for j in new_ner:
        for k in j:
            
            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp
print(len(med7_ner_data))

21837


In [12]:
pd.to_pickle(med7_ner_data, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner_word_dict.pkl")

In [13]:
def mean(a):
    return sum(a) / len(a)

In [20]:
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(w2vec[i[0]])
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    t = np.asarray(map(mean, zip(*avg)))
                    patient_temp.append(t)
        if len(patient_temp) == 0: continue
        new_word2vec[k] = patient_temp
    print(len(new_word2vec))
    #############################################################################
    print("fasttext starting..")
        
    new_fasttextvec = {}

    for k,v in data.items():

        patient_temp = []

        for i in v:
            try:
                patient_temp.append(fasttext[i[0]])
            except:
                pass
        if len(patient_temp) == 0: continue
        new_fasttextvec[k] = patient_temp
    print(len(new_fasttextvec))
    #############################################################################    
        
    print("combined starting..")
    new_concatvec = {}

    for k,v in data.items():
        patient_temp = []
        #if k != 6: continue
        for i in v:
            w2vec_temp = []
            try:
                w2vec_temp = w2vec[i[0]]
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: 
                        w2vec_temp = [0] * 100
                    else:
                        avg = np.asarray(avg)
                        w2vec_temp = np.asarray(map(mean, zip(*avg)))
                else:
                    w2vec_temp = [0] * 100

            #fasttemp = fasttext[i[0]]

            #appended = np.append(fasttemp, w2vec_temp, 0)
            appended = np.append(fasttext[i[0]], w2vec_temp, 0)
            patient_temp.append(appended)
        if len(patient_temp) == 0: continue
        new_concatvec[k] = patient_temp

    print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))
    pd.to_pickle(new_word2vec, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+names+"_word2vec_dict.pkl")
    pd.to_pickle(new_fasttextvec, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+names+"_fasttext_dict.pkl")
    pd.to_pickle(new_concatvec, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+names+"_combined_dict.pkl")

w2vec starting..
21731
fasttext starting..
21809
combined starting..


KeyError: ignored

In [ ]:
diff = set(new_fasttext_dict.keys()).difference(set(new_word2vec_dict))
for i in diff:
    del new_fasttext_dict[i]
    del new_combined_dict[i]
print (len(new_word2vec_dict), len(new_fasttext_dict), len(new_combined_dict))


pd.to_pickle(new_word2vec_dict, "data/"+"new_ner"+"_word2vec_limited_dict.pkl")
pd.to_pickle(new_fasttext_dict, "data/"+"new_ner"+"_fasttext_limited_dict.pkl")
pd.to_pickle(new_combined_dict, "data/"+"new_ner"+"_combined_limited_dict.pkl")